In [5]:
df = pd.read_csv("metadata.csv")
df.describe()

In [4]:
rdf = pd.DataFrame(df["Class"].value_counts())
print(rdf)

In [11]:


# Calculate the order of classes based on the mean of 'Vertices'
class_order = df.groupby('Class')['Vertices'].mean().sort_values().index

# Create the side-by-side boxplots with ordered classes
plt.figure(figsize=(15, 10))
sns.boxplot(x='Class', y='Cells', data=df, palette=['#1f77b4', '#ff7f0e'], order=class_order)
plt.title('Vertices by Class (Ordered by Average Vertices)')
plt.xlabel('Class')
plt.ylabel('Count of Vertices')
plt.xticks(rotation=90)  # Rotate class labels for better readability
plt.legend(title='Source')
plt.show()

In [80]:
data = pd.DataFrame(data={"Vertices":df["Vertices"],"Cells":df["Cells"]})


plt.figure(figsize=(15, 10))
sns.scatterplot(x='Vertices', y='Cells', data=data)
plt.plot([1, 1], [120000, 120000],linewidth=2)

plt.title('Vertex to Cell Relation')
plt.xlabel('Vertices')
plt.ylabel('Cells')
plt.xticks(rotation=90)  # Rotate class labels for better readability
plt.legend(title='Source')
plt.plot([0, 60000], [0, 120000], linewidth=2, label="1:2")
plt.plot([0, 20000], [0, 120000], linewidth=2, label="1:6")
plt.plot([0, 120000], [0, 40000], linewidth=2, label="3:1")

plt.legend()
plt.show()

In [36]:

plt.figure(figsize=(8, 6))
sns.scatterplot(x='ground_truth', y='sampling', data=rdf)
plt.title('Diagonal Heuristic Evaluation')
plt.xlabel('Brute Force')
plt.ylabel('Heuristic')
plt.plot([0, np.sqrt(3)], [0, np.sqrt(3)], linewidth=2, label="expected",color="green")


In [28]:
dfmanifest = pd.read_csv("shape_manifest.csv")
dfmanifest["Path"]=dfmanifest["Path"].apply(func=(lambda x: x.replace("\\","/")))
df["Path"]=df["Path"].apply(func=(lambda x: x.replace("\\","/")))
df_sub = df[["Vertices","Path","Cells"]]


In [48]:
other_data = pd.DataFrame(data={"Vertices":df["Vertices"],"Cells":df["Cells"],"Path":df["Path"]})
other_data.set_index("Path")
joined_data = failed_meta.merge(other_data,on=["Path","Cells","Vertices"],how="right")

joined_data.loc[pd.isna(joined_data['Color']),'Color'] = "Blue"
print(joined_data.head())

In [73]:
mesh_features = pd.read_csv("mesh_features.csv")

print("D00348.obj" in mesh_features["mesh_name"].values)
print(mesh_features.loc[mesh_features["mesh_name"]=="D00348.obj"])